# SoftRobot-Control

## Programming methods for Arduino via python

### 1) Prigramming via firmata protocol

In [ ]:
# before using lib pyfirmata via python, the standardfirmata.ino should be uploaded to Arduino board to ensure the protocol.

import pyfirmata                     # alternative way for the serial lib
import numpy as np
import time

# we decide to choose ArduinoMega due to more PWM pins
port = 'COM5'                        # select the port
board = pyfirmata.ArduinoMega(port)  # define the board
sleep(2)                             # wait for the analog information              

## configure the serial port for vacuum pressure
P1 = board.get_pin('a:1:i')
P2 = board.get_pin('a:2:i')
P3 = board.get_pin('a:3:i')
P4 = board.get_pin('a:4:i')
P5 = board.get_pin('a:5:i')

## start the iterator process, to avoid overflow from the serial port
it = pyfirmata.util.Iterator(board)
it.start()

try:
    while True:
        ## Data read and calibrate
        Pv1 = P1.read()
#         PV1 = Pv1*5*23-115   # output via pyfirmata is between 0 and 1
        Pv2 = P2.read()
#         PV2 = Pv2*5*23-115
        Pv3 = P3.read()
#         PV3 = Pv3*5*23-115
        Pv4 = P4.read()
#         PV4 = Pv4*5*23-115
        Pv5 = P5.read()
#         PV5 = Pv5*5*23-115
        print('P1=',Pv1,'P2=',Pv2,'P3=',Pv3,'P4=',Pv4,'P5=',Pv5)
except KeyboardInterrupt:    # Capture expections, like: Crtl+C 
    board.exit()

### 2) Programming via pyserial

In [ ]:
import serial
port = serial.Serial('COM4', 9600, timeout=1)

## serial port information should be programmed in Arduino IDE for pressure information
while True:
    print port.readline()

port.close()

### 3) Programming via Firmata and GUI

In [ ]:
## manually configure the different conditions for the programs, meanwhile record the responses

import pyfirmata
from tkinter import *
import matplotlib.pyplot as plt

port = 'COM5'    # select the port
board = pyfirmata.ArduinoMega(port)  # define the board
sleep(2)         # wait for the buffer of analog input

# Initialize main windows with title and size 
PV = tkinter.Tk() 
PV.title("Vacuum Pressure") 
PV.minsize(500,90)
# Label widget 
PVlabel = tkinter.Label(top, text = "") 
PVlabel.pack()
# Start and open the window 
top.mainloop()

### 4） Programming via firmata and save data

In [ ]:
# use firmata to control the board and the sensor data can be saved in CSV file
## This program is not for real-time data visualization

import pyfirmata
from time import sleep
import csv

port = 'COM5'
board = pyfirmata.ArduinoMega(port)
it = pyfirmata.util.Iterator(board)
it.start()

P1 = board.get_pin('a:1:i')

with open('PressureData.csv','w+') as f:
    w = csv.writer(f)
    w.writerow(["Number","Value"])
    i = 0
    Pv1 = P1.read()
    PV1 = 115*Pv1-115    # Data calibration for pressure sensor
    while i < 10:
        sleep() # Sleep time is in second scale
        if Pv1 is not None:
            i += 1
            row = [i,PV1]
            w.writerow(row)
    print("CSV file is done!")

board.exit()

### 5) Programming via matplotlib for the analog output

In [ ]:
# use firmata to control the board and the sensor data can be saved in CSV file
# Data visualization: the sensor/CSV data can be plotted in real-time

import pyfirmata
from time import sleep
import numpy as np
import matplotlib.pyplot as plt
import sys, csv

# Configure the port and board with firmata protocol
port = 'COM4'
board = pyfirmata.ArduinoMega(port)
# Use iterator thread to avoid buffer overflow
it = pyfirmata.util.Iterator(board)
it.start()

# configure the pinMode for different components
P1 = board.get_pin('a:1:i')

# Initialize the interactive mode
plt.ion()

board.exit()

### 6) Recording with the camera

In [ ]:
import cv2
import tkinter
import PIL.Image, PIL.ImageTk
import time

class Application:
    def __init__(self, window, window_title, video_source):
        self.window = window
        self.window.title(window_title)
        self.SRvideo = cv2.VideoCapture(video_source)
        if not self.SRvideo.isOpened():
            raise ValueError("Unable to open video source", video_source)
        # the size of HOZAN USB camera is 640*480
        self.width = self.SRvideo.get(3)   # cv2.CAP_PROP_FRAME_WIDTH = 3
        self.height = self.SRvideo.get(4) # cv2.CAP_PROP_FRAME_HEIGHT = 4
        
        # Canvas configuration: inlucding video frame, button and pressure sensor curves
        self.canvas = tkinter.Canvas(window, width = 2*self.width, height = self.height)
        self.canvas.pack()
        self.btn_snapshot=tkinter.Button(window, text="Snapshot", width=50, command=self.snapshot)
        self.btn_snapshot.pack(anchor=tkinter.CENTER, expand=True)
#         self.
        
#         SRwri = cv2.VideoWriter('SRcrawling.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (int(self.width),int(self.height)))
        # Updata the frame every 10 milliseconds
        self.delay = 10
        self.update()
        self.window.mainloop()
        
    # function for snapshot 
    def snapshot(self):
        ret, frame = self.SRvideo.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if ret:
            cv2.imwrite("SRcrawling-" + time.strftime("%Y-%m-%d-%H-%M-%S") + ".jpg", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    # function for the frame update
    def update(self):
        ret, frame = self.SRvideo.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if ret:
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = tkinter.NW)
        self.window.after(self.delay, self.update)
            
# create a window and pass it to the Application object
Application(tkinter.Tk(), "SRcrawling", 1)
# SRwri = cv2.VideoWriter('SRcrawling.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (Application.width, Application.height))